<a href="https://colab.research.google.com/github/jessilver/Organizacao_Computadores/blob/develop/Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Faça uma cópia no seu drive

Importante: as instalações precisam ser feita toda vez que for iniciada uma nova seção no Colab

# Jessé Eliseu Nunes da Silva

# Como usar:

As implementações foram separadas e partes, para que a interface reconheça a parte que desesja utilizar bastas "ativa-la", trocando o valor da variavel de status de false para true.

## Testes:

### Teste Parte 1:

```
# --- Teste da Parte 1: Extensões e Erros ---

# 1. Teste de Divisão de Inteiros (DIV)
ADDI x1, x0, 40
ADDI x2, x0, 5
DIV x3, x1, x2  # Deve resultar em 8

# 2. Teste de Overflow (deve reportar ERRO e continuar)
ADDI x5, x0, 2147483647 # Valor máximo de inteiro 32-bit
ADDI x6, x0, 100
ADD x7, x5, x6  # Esta operação vai causar overflow

# 3. Teste de Divisão por Zero (deve reportar ERRO e continuar)
DIV x8, x1, x0 # x0 é sempre zero

# 4. Teste de Ponto Flutuante (deve executar após os erros)
# Usando os bits de 4.5 (1083179008) e 2.0 (1073741824)
ADDI x10, x0, 1083179008 # Carrega bits de 4.5
ADDI x11, x0, 1073741824 # Carrega bits de 2.0
ADD.S x12, x10, x11 # Soma de floats -> 6.5
MUL.S x13, x10, x11 # Multiplicação de floats -> 9.0
```

### Teste Parte 2:

```
# --- Teste da Parte 2: Funcionalidades Base ---

# Setup: Coloca valores iniciais nos registradores usando ADDI.
# (Usamos ADDI por conveniência, mas o foco do teste são as instruções abaixo)
ADDI x1, x0, 15
ADDI x2, x0, 10
ADDI x3, x0, 3

# 1. Soma os dois primeiros valores
# Requisito: ADD
ADD x4, x1, x2

# 2. Multiplica o resultado pelo terceiro valor
# Requisito: MUL
MUL x5, x4, x3

# 3. Armazena o resultado final (em x5) no endereço de memória 20
# Requisito: STORE (SW)
SW x5, 20

# 4. Carrega o valor do endereço de memória 20 para outro registrador (x6) para verificação
# Requisito: LOAD (LW)
LW x6, 20

# 5. Usa SUB para uma verificação final
# Requisito: SUB
SUB x7, x6, x5
```

### Teste Parte 3:

```
# --- Teste da Parte 3: Pipeline Básico ---

# Sequência de 5 instruções independentes para
# observar o preenchimento e esvaziamento do pipeline.

ADDI x1, x0, 1
ADDI x2, x0, 2
ADDI x3, x0, 3
ADDI x4, x0, 4
ADDI x5, x0, 5
```

### Teste Parte 4

```
# --- Teste da Parte 4: Data Hazards ---

# 1. Hazard direto (ADD -> SUB)
ADDI x1, x0, 10
ADDI x2, x0, 20
ADD  x3, x1, x2  # x3 é escrito aqui
SUB  x4, x3, x5  # x3 é lido imediatamente aqui -> HAZARD!

# 2. Hazard de LW (Load-Use)
SW   x1, 30      # Armazena 10 no endereço 30
LW   x6, 30      # x6 é escrito com o valor da memória aqui
ADD  x7, x6, x1  # x6 é lido imediatamente aqui -> HAZARD de LW!
```

### Teste Parte 5

```
# --- Teste da Parte 5: Cache e Hierarquia de Memória ---

# 1. Primeiro acesso a um endereço: MISS
# Escreve 50 no endereço 10
ADDI x1, x0, 50
SW x1, 10

# 2. Acesso ao mesmo endereço: HIT (localidade temporal)
LW x2, 10

# 3. Acesso a um endereço próximo: pode ser HIT ou MISS dependendo do mapeamento
SW x1, 11

# 4. Loop para forçar hits (localidade temporal e espacial)
# Lê 4 vezes do mesmo endereço (deve ser 1 miss e 3 hits)
ADDI x5, x0, 4   # Contador
ADDI x6, x0, 20  # Endereço
loop_start:
LW x7, 20
ADDI x5, x5, -1
BNE x5, x0, loop_start # (NOTA: BNE não está implementado, isso é conceitual)

# 5. Acesso a um endereço que causa conflito (pode remover outro da cache)
# Se a cache tem 8 conjuntos, o endereço 18 (18 % 8 = 2) e 10 (10 % 8 = 2)
# mapeiam para o mesmo conjunto. Este acesso pode expulsar o bloco do endereço 10.
SW x1, 18

# 6. Reacesso ao endereço 10: agora será um MISS de novo (miss de conflito)
LW x3, 10
```

# Interface

## Código

In [179]:
# ===================================================================
# Bloco 1: Interface do Usuário
# VERSÃO FINAL CORRIGIDA: Corrige o AttributeError na inicialização da UI.
# ===================================================================

import ipywidgets as widgets
from IPython.display import display, clear_output

class LogColors: INFO, SUCCESS, ERROR, WARNING, RESET = ('\033[94m', '\033[92m', '\033[91m', '\033[93m', '\033[0m')

# --- Widgets ---
code_area = widgets.Textarea(value="", placeholder="Escreva seu código assembly aqui...", description="Code:", layout=widgets.Layout(width="100%", height="200px"))
output_area = widgets.Output()

part_selector = widgets.RadioButtons(options=[('Ciclo Único (Partes 1 & 2)', 'p1_2'), ('Pipeline (Partes 3, 4 & 5)', 'p3_5')], description='Simular:')
hazard_mode_selector = widgets.RadioButtons(options=[('Sem tratamento de hazards', 'none'), ('Stall em hazards', 'stall'), ('Forwarding (Bypass)', 'forwarding')], value='forwarding', description='Hazards:')
cache_selector = widgets.RadioButtons(options=[('Com Cache', 'enabled'), ('Sem Cache (Acesso Direto)', 'disabled')], value='enabled', description='Memória:')

run_button = widgets.Button(description="Run Program", button_style="success")
reg_button = widgets.Button(description="Show Registers", button_style="info")
mem_button = widgets.Button(description="Show Memory", button_style="info")
cache_button = widgets.Button(description="Show Cache Stats", button_style="warning")
start_addr = widgets.IntText(value=0, description="Start Addr")
mem_range = widgets.IntText(value=10, description="Range")

# --- Handlers ---
def run_program_handler(b):
    with output_area:
        clear_output(wait=True)
        print(f"{LogColors.INFO}Inicializando hardware virtual...{LogColors.RESET}")

    global regs, mem, cache
    regs = RegisterFile(); mem = SimpleMemory(64); cache = Cache(num_sets=8, main_memory=mem)
    code = code_area.value

    clear_output(wait=True); display(ui)
    print(f"{LogColors.INFO}--- Log de Execução ---{LogColors.RESET}")

    selected_part = part_selector.value
    hazard_mode = hazard_mode_selector.value
    cache_mode = cache_selector.value
    part_label = next((label for label, value in part_selector.options if value == selected_part), "N/A")
    hazard_label = next((label for label, value in hazard_mode_selector.options if value == hazard_mode), "N/A")
    cache_label = next((label for label, value in cache_selector.options if value == cache_mode), "N/A")

    print(f"{LogColors.INFO}Modo de Simulação: {part_label}{LogColors.RESET}")
    if selected_part == 'p3_5':
        print(f"{LogColors.INFO}Tratamento de Hazard: {hazard_label}{LogColors.RESET}")
        print(f"{LogColors.INFO}Hierarquia de Memória: {cache_label}{LogColors.RESET}")
    print()

    if selected_part == 'p1_2':
        run_program_single_cycle(code, regs, mem)
    elif selected_part == 'p3_5':
        run_program_pipeline(code, regs, mem, cache, hazard_mode=hazard_mode, cache_mode=cache_mode)

def reg_button_handler(b): show_registers(regs)
def mem_button_handler(b): show_memory(mem, start=start_addr.value, rng=mem_range.value)
def cache_button_handler(b):
    if part_selector.value != 'p3_5' or cache_selector.value == 'disabled':
        with output_area: clear_output(wait=True); print(f"{LogColors.WARNING}A cache não está ativa. As estatísticas não são aplicáveis.{LogColors.RESET}")
    else: show_cache_stats(cache)

# --- Lógica de UI Dinâmica (FUNÇÃO CORRIGIDA) ---
def on_part_selection_change(change):
    # CORREÇÃO: Acessa o valor com a sintaxe de dicionário ['new'],
    # que funciona tanto para o objeto 'change' do widget quanto
    # para o dicionário passado manualmente.
    is_pipeline = (change['new'] == 'p3_5')

    hazard_mode_selector.layout.display = 'flex' if is_pipeline else 'none'
    cache_selector.layout.display = 'flex' if is_pipeline else 'none'
    cache_button.layout.display = 'inline-block' if is_pipeline else 'none'

part_selector.observe(on_part_selection_change, names='value')

# --- Conexões e Layout ---
run_button.on_click(run_program_handler)
reg_button.on_click(reg_button_handler)
mem_button.on_click(mem_button_handler)
cache_button.on_click(cache_button_handler)

regs, mem, cache = None, None, None
top_controls = widgets.HBox([run_button, reg_button, mem_button, cache_button, start_addr, mem_range])
pipeline_options = widgets.VBox([hazard_mode_selector, cache_selector])
ui = widgets.VBox([part_selector, pipeline_options, code_area, top_controls, output_area])

# Inicializa o estado da UI (chamada manual)
on_part_selection_change({'new': part_selector.value})

AttributeError: 'dict' object has no attribute 'new'

## Execução

In [ ]:
display(ui)

# Componetes Comuns

In [ ]:
# ===================================================================
# Bloco 2: Componentes Comuns
# Contém as classes, funções de formatação e de exibição que são
# usadas por todas as partes do simulador.
# ===================================================================

import pandas as pd
import struct

# --- Classe de Cores para o Log ---
class LogColors:
    INFO = '\033[94m'
    SUCCESS = '\033[92m'
    ERROR = '\033[91m'
    WARNING = '\033[93m'
    RESET = '\033[0m'

# --- Classes do Simulador ---
class SimpleMemory:
    # ... (código da classe inalterado)
    def __init__(self, size_in_words=64):
        self.size = size_in_words
        self.mem = [0] * size_in_words
    def load_word(self, address):
        if 0 <= address < self.size: return self.mem[address]
        raise IndexError(f"Endereço de memória inválido: {address}")
    def store_word(self, address, value):
        if 0 <= address < self.size: self.mem[address] = value & 0xFFFFFFFF
        else: raise IndexError(f"Endereço de memória inválido: {address}")

class RegisterFile:
    # ... (código da classe inalterado)
    def __init__(self):
        self.regs = [0] * 32
    def read(self, reg_num):
        if 0 <= reg_num < 32: return self.regs[reg_num]
        raise IndexError(f"Número de registrador inválido: x{reg_num}")
    def write(self, reg_num, value):
        if reg_num == 0: return
        if 0 < reg_num < 32: self.regs[reg_num] = value & 0xFFFFFFFF
        else: raise IndexError(f"Número de registrador inválido: x{reg_num}")

# --- Funções Auxiliares ---
def float_to_int_bits(f): return struct.unpack('I', struct.pack('f', float(f)))[0]
def int_bits_to_float(i):
    try: return struct.unpack('f', struct.pack('I', i & 0xFFFFFFFF))[0]
    except: return float('nan')
def to_signed32(value):
    value &= 0xFFFFFFFF
    return value - (1 << 32) if value & (1 << 31) else value
def format_instruction(instr):
    # ... (código da função inalterado)
    if not instr: return ""
    op = instr['op']
    if op in ["ADD", "SUB", "MUL", "DIV", "ADD.S", "MUL.S"]: return f"{op:<8} x{instr['rd']}, x{instr['rs1']}, x{instr['rs2']}"
    if op == "ADDI": return f"{op:<8} x{instr['rd']}, x{instr['rs1']}, {instr['imm']}"
    if op == "LW": return f"{op:<8} x{instr['rd']}, {instr['addr']}"
    if op == "SW": return f"{op:<8} x{instr['rs1']}, {instr['addr']}"
    return str(instr)

# --- Funções de Exibição ---
def show_registers(regs):
    # ... (código da função inalterado)
    output_widgets = []
    for i in range(0, 32, 8):
        start_reg, end_reg = i, i + 8
        register_values = [regs.read(j) for j in range(start_reg, end_reg)]
        data = {"Reg": [f"x{j}" for j in range(start_reg, end_reg)],
                "Valor Int": [to_signed32(val) for val in register_values],
                "Valor FP": [f"{int_bits_to_float(val):.4f}" for val in register_values]}
        df = pd.DataFrame(data).set_index("Reg")
        out = widgets.Output()
        with out: display(df)
        output_widgets.append(out)
    with output_area:
        clear_output(wait=True)
        display(widgets.HBox(output_widgets))

def show_memory(mem, start=0, rng=10):
    try:
        data = {"Endereço": list(range(start, start + rng)), "Valor": [mem.load_word(i) for i in range(start, start + rng)]}
        df = pd.DataFrame(data)
        with output_area: clear_output(wait=True); display(df)
    except IndexError as e:
        with output_area:
            clear_output(wait=True)
            print(f"{LogColors.ERROR}Erro ao acessar memória: {e}{LogColors.RESET}")

# Parte 2

In [180]:
# ===================================================================
# Bloco 3: Lógica da Parte 2 (Simulador de Ciclo Único - Base)
# ===================================================================
# ... (O código deste bloco permanece o mesmo das respostas anteriores)
def execute_instruction_single_cycle(instr, regs, mem):
    op = instr["op"]
    if op in ["ADD", "SUB", "MUL"]:
        rs1_val, rs2_val = to_signed32(regs.read(instr["rs1"])), to_signed32(regs.read(instr["rs2"]))
        if op == "ADD": result = rs1_val + rs2_val
        elif op == "SUB": result = rs1_val - rs2_val
        else: result = rs1_val * rs2_val
        regs.write(instr["rd"], result)
    elif op == "LW": regs.write(instr["rd"], mem.load_word(instr["addr"]))
    elif op == "SW": mem.store_word(instr["addr"], regs.read(instr["rs1"]))
    else: raise ValueError(f"Instrução '{op}' não pertence ao conjunto base da Parte 2.")

def parse_program_single_cycle(asm_code):
    program = []
    for line in asm_code.strip().splitlines():
        line = line.strip().split('#')[0]
        if not line: continue
        parts = line.replace(",", "").split()
        op = parts[0].upper()
        if op in ["ADD", "SUB", "MUL"]: instr = {"op": op, "rd": int(parts[1][1:]), "rs1": int(parts[2][1:]), "rs2": int(parts[3][1:])}
        elif op == "LW": instr = {"op": op, "rd": int(parts[1][1:]), "addr": int(parts[2])}
        elif op == "SW": instr = {"op": op, "rs1": int(parts[1][1:]), "addr": int(parts[2])}
        else: raise ValueError(f"Instrução '{op}' não reconhecida pelo parser base.")
        program.append(instr)
    return program

def run_program_single_cycle(asm_code, regs, mem):
    try: program = parse_program_single_cycle(asm_code)
    except ValueError as e: print(f"{LogColors.ERROR}ERRO DE SINTAXE: {e}{LogColors.RESET}"); return
    for cycle, instr in enumerate(program, 1):
        formatted_instr = format_instruction(instr)
        try:
            execute_instruction_single_cycle(instr, regs, mem)
            print(f"{LogColors.INFO}Ciclo {cycle}:{LogColors.RESET} {formatted_instr:<25} -> {LogColors.SUCCESS}Sucesso!{LogColors.RESET}")
        except (ValueError, IndexError, OverflowError) as e:
            print(f"{LogColors.INFO}Ciclo {cycle}:{LogColors.RESET} {formatted_instr:<25} -> {LogColors.ERROR}ERRO: {e}{LogColors.RESET}")

print(f"{LogColors.SUCCESS}Funções base de simulação (Parte 2) carregadas.{LogColors.RESET}")

Funções base de simulação (Parte 2) carregadas.


# Parte 1


In [181]:
# ===================================================================
# Bloco 4: Lógica da Parte 1 (Extensões para o Simulador)
# ===================================================================
# ... (O código deste bloco permanece o mesmo das respostas anteriores)
def execute_instruction_single_cycle(instr, regs, mem):
    op = instr["op"]
    INT32_MIN, INT32_MAX = -2**31, 2**31 - 1
    if op in ["ADD", "ADDI", "SUB", "MUL", "DIV"]:
        rs1_val = to_signed32(regs.read(instr["rs1"]))
        rs2_val = instr["imm"] if op == "ADDI" else to_signed32(regs.read(instr["rs2"]))
        op_symbol = {'ADD': '+', 'ADDI': '+', 'SUB': '-', 'MUL': '*', 'DIV': '/'}[op]
        if op in ["ADD", "ADDI"]: result = rs1_val + rs2_val
        elif op == "SUB": result = rs1_val - rs2_val
        elif op == "MUL": result = rs1_val * rs2_val
        elif op == "DIV":
            if rs2_val == 0: raise ValueError("Divisão por zero")
            if rs1_val == INT32_MIN and rs2_val == -1: raise OverflowError(f"Overflow na divisão")
            result = int(rs1_val / rs2_val)
        if not (INT32_MIN <= result <= INT32_MAX) and op != "DIV": raise OverflowError(f"Overflow na operação: {rs1_val} {op_symbol} {rs2_val}")
        regs.write(instr["rd"], result)
    elif op in ["ADD.S", "MUL.S"]:
        float1, float2 = int_bits_to_float(regs.read(instr["rs1"])), int_bits_to_float(regs.read(instr["rs2"]))
        result_float = (float1 + float2) if op == "ADD.S" else (float1 * float2)
        regs.write(instr["rd"], float_to_int_bits(result_float))
    elif op == "LW": regs.write(instr["rd"], mem.load_word(instr["addr"]))
    elif op == "SW": mem.store_word(instr["addr"], regs.read(instr["rs1"]))
    else: raise ValueError(f"Instrução desconhecida {op}")

def parse_program_single_cycle(asm_code):
    program = []
    for line in asm_code.strip().splitlines():
        line = line.strip().split('#')[0]
        if not line: continue
        parts = line.replace(",", "").split()
        op = parts[0].upper()
        if op in ["ADD", "SUB", "MUL", "DIV", "ADD.S", "MUL.S"]: instr = {"op": op, "rd": int(parts[1][1:]), "rs1": int(parts[2][1:]), "rs2": int(parts[3][1:])}
        elif op == "ADDI": instr = {"op": op, "rd": int(parts[1][1:]), "rs1": int(parts[2][1:]), "imm": int(parts[3])}
        elif op == "LW": instr = {"op": op, "rd": int(parts[1][1:]), "addr": int(parts[2])}
        elif op == "SW": instr = {"op": op, "rs1": int(parts[1][1:]), "addr": int(parts[2])}
        else: raise ValueError(f"Instrução desconhecida: {line}")
        program.append(instr)
    return program

print(f"{LogColors.SUCCESS}Funções de simulação da Parte 1 e 2 (versão completa) carregadas.{LogColors.RESET}")

Funções de simulação da Parte 1 e 2 (versão completa) carregadas.


# Parte 3

In [182]:
# ===================================================================
# Bloco 5: Lógica da Parte 3 (Simulador de Pipeline - Base)
# ===================================================================
# ... (O código deste bloco permanece o mesmo das respostas anteriores)
def run_program_pipeline(asm_code, regs, mem, cache, hazard_mode='none'): # Adicionado 'cache' para consistência
    try: program = parse_program_single_cycle(asm_code)
    except ValueError as e: print(f"{LogColors.ERROR}ERRO DE SINTAXE: {e}{LogColors.RESET}"); return
    pc, cycle = 0, 0
    pipeline_stages = {'IF': None, 'ID': None, 'EX': None, 'MEM_WB': None}
    history = []
    while pc < len(program) or any(s is not None for s in pipeline_stages.values()):
        cycle += 1
        prev_stages = pipeline_stages.copy()
        pipeline_stages['MEM_WB'] = prev_stages['EX']
        pipeline_stages['EX'] = prev_stages['ID']
        pipeline_stages['ID'] = prev_stages['IF']
        if pc < len(program): pipeline_stages['IF'] = program[pc]; pc += 1
        else: pipeline_stages['IF'] = None
        history.append({ 'Ciclo': cycle, 'IF': format_instruction(pipeline_stages['IF']), 'ID': format_instruction(pipeline_stages['ID']), 'EX': format_instruction(pipeline_stages['EX']), 'MEM_WB': format_instruction(pipeline_stages['MEM_WB']) })
        if cycle > len(program) + 10: print(f"{LogColors.WARNING}Simulação parada: limite de ciclos.{LogColors.RESET}"); break
    print(f"\n{LogColors.INFO}--- Tabela de Execução do Pipeline ---{LogColors.RESET}")
    if history:
        df = pd.DataFrame(history).set_index('Ciclo')
        with output_area: display(df.fillna(""))
    print(f"\n{LogColors.INFO}Total de Ciclos (Pipeline):{LogColors.RESET} {cycle}\n{LogColors.INFO}Total de Ciclos (Ciclo Único seria):{LogColors.RESET} {len(program)}")

print(f"{LogColors.SUCCESS}Funções base de simulação de Pipeline (Parte 3) carregadas.{LogColors.RESET}")

Funções base de simulação de Pipeline (Parte 3) carregadas.


# Parte 4

In [183]:
# ===================================================================
# Bloco 6: Lógica da Parte 4 (Pipeline com Hazards)
# ===================================================================
# ... (O código deste bloco permanece o mesmo das respostas anteriores)
def run_program_pipeline(asm_code, regs, mem, cache, hazard_mode='forwarding'): # Adicionado 'cache'
    try: program = parse_program_single_cycle(asm_code)
    except ValueError as e: print(f"{LogColors.ERROR}ERRO DE SINTAXE: {e}{LogColors.RESET}"); return
    pc, cycle, stalls = 0, 0, 0
    pipeline_stages = {'IF': None, 'ID': None, 'EX': None, 'MEM_WB': None}
    history, hazard_logs = [], []
    while pc < len(program) or any(s is not None for s in pipeline_stages.values()):
        cycle += 1
        prev_stages = pipeline_stages.copy()
        stall_this_cycle = False
        instr_wb = prev_stages['MEM_WB']
        if instr_wb:
            if 'result' in instr_wb: regs.write(instr_wb['rd'], instr_wb['result'])
            elif instr_wb['op'] == 'SW': mem.store_word(instr_wb['addr'], regs.read(instr_wb['rs1']))
        instr_ex = prev_stages['EX']
        if instr_ex:
            op = instr_ex['op']
            val1 = regs.read(instr_ex.get('rs1', 0))
            val2 = regs.read(instr_ex.get('rs2', 0)) if 'rs2' in instr_ex else instr_ex.get('imm', 0)
            if hazard_mode == 'forwarding':
                instr_mem = prev_stages['MEM_WB']
                if instr_mem and 'result' in instr_mem and 'rd' in instr_mem:
                    fwd_reg = instr_mem['rd']
                    if fwd_reg == instr_ex.get('rs1'): val1 = instr_mem['result']; hazard_logs.append(f"{LogColors.INFO}Ciclo {cycle}:{LogColors.RESET} {LogColors.SUCCESS}FORWARDING (MEM->EX){LogColors.RESET} para reg x{fwd_reg} na instrução '{format_instruction(instr_ex)}'")
                    if 'rs2' in instr_ex and fwd_reg == instr_ex.get('rs2'): val2 = instr_mem['result']; hazard_logs.append(f"{LogColors.INFO}Ciclo {cycle}:{LogColors.RESET} {LogColors.SUCCESS}FORWARDING (MEM->EX){LogColors.RESET} para reg x{fwd_reg} na instrução '{format_instruction(instr_ex)}'")
            if op in ["ADD", "ADDI"]: instr_ex['result'] = to_signed32(val1) + to_signed32(val2)
            elif op == "SUB": instr_ex['result'] = to_signed32(val1) - to_signed32(val2)
            elif op == "LW": instr_ex['result'] = mem.load_word(instr_ex['addr'])
            pipeline_stages['MEM_WB'] = instr_ex
        else: pipeline_stages['MEM_WB'] = None
        instr_id = prev_stages['ID']
        if instr_id:
            src_regs = {instr_id.get('rs1'), instr_id.get('rs2')} - {None}
            instr_ex_prev = prev_stages['EX']
            if instr_ex_prev and 'rd' in instr_ex_prev and instr_ex_prev['rd'] in src_regs:
                hazard_reg = instr_ex_prev['rd']
                if instr_ex_prev['op'] == 'LW':
                    if hazard_mode != 'none': stall_this_cycle = True; hazard_logs.append(f"{LogColors.INFO}Ciclo {cycle}:{LogColors.RESET} {LogColors.ERROR}HAZARD DE LOAD-USE detectado{LogColors.RESET} no reg x{hazard_reg}. {LogColors.WARNING}Ação: STALL OBRIGATÓRIO.{LogColors.RESET}")
                    else: hazard_logs.append(f"{LogColors.INFO}Ciclo {cycle}:{LogColors.RESET} {LogColors.ERROR}HAZARD DE LOAD-USE detectado{LogColors.RESET} no reg x{hazard_reg}. {LogColors.WARNING}Ação: Nenhuma.{LogColors.RESET}")
                else:
                    if hazard_mode == 'stall': stall_this_cycle = True; hazard_logs.append(f"{LogColors.INFO}Ciclo {cycle}:{LogColors.RESET} {LogColors.WARNING}HAZARD DE DADOS (EX->ID) detectado{LogColors.RESET} no reg x{hazard_reg}. {LogColors.WARNING}Ação: STALL.{LogColors.RESET}")
                    elif hazard_mode == 'none': hazard_logs.append(f"{LogColors.INFO}Ciclo {cycle}:{LogColors.RESET} {LogColors.ERROR}HAZARD DE DADOS (EX->ID) detectado{LogColors.RESET} no reg x{hazard_reg}. {LogColors.WARNING}Ação: Nenhuma.{LogColors.RESET}")
        if stall_this_cycle: stalls += 1; pipeline_stages['EX'] = None
        else:
            pipeline_stages['EX'] = prev_stages['ID']
            pipeline_stages['ID'] = prev_stages['IF']
            if pc < len(program): pipeline_stages['IF'] = program[pc]; pc += 1
            else: pipeline_stages['IF'] = None
        history.append({ 'Ciclo': cycle, 'IF': format_instruction(pipeline_stages['IF']), 'ID': format_instruction(pipeline_stages['ID']), 'EX': f"*{format_instruction(pipeline_stages['EX'])}" if stall_this_cycle else format_instruction(pipeline_stages['EX']), 'MEM_WB': format_instruction(pipeline_stages['MEM_WB'])})
        if cycle > (len(program) + stalls) * 2 and cycle > 20: print(f"{LogColors.WARNING}Simulação parada.{LogColors.RESET}"); break
    if hazard_logs: print(f"\n{LogColors.INFO}--- Detalhes de Hazards e Forwarding ---{LogColors.RESET}"); [print(log) for log in hazard_logs]
    print(f"\n{LogColors.INFO}--- Tabela de Execução do Pipeline ---{LogColors.RESET}")
    if history: df = pd.DataFrame(history).set_index('Ciclo'); display(df.fillna(""))
    print(f"\n{LogColors.INFO}Total de Ciclos (Pipeline):{LogColors.RESET} {cycle}\n{LogColors.INFO}Número de Stalls (bolhas):{LogColors.RESET} {stalls}\n{LogColors.INFO}Total de Ciclos (Ciclo Único seria):{LogColors.RESET} {len(program)}")

print(f"{LogColors.SUCCESS}Funções de simulação de Pipeline com Hazards (Parte 4) carregadas.{LogColors.RESET}")

Funções de simulação de Pipeline com Hazards (Parte 4) carregadas.


# Parte 5

In [184]:
# ===================================================================
# Bloco 7: Lógica da Parte 5 (Hierarquia de Memória - Cache)
# Este bloco define a classe da Cache e suas funcionalidades.
# ===================================================================

class Cache:
    def __init__(self, num_sets, main_memory):
        self.num_sets = num_sets
        self.main_memory = main_memory
        self.sets = [[self._create_cache_line(), self._create_cache_line()] for _ in range(num_sets)]
        self.hits = 0
        self.misses = 0

    def _create_cache_line(self):
        return {'valid': 0, 'tag': 0, 'data': 0, 'lru': 0} # lru=0 -> Way 0, lru=1 -> Way 1

    def _get_address_parts(self, address):
        index = address % self.num_sets
        tag = address // self.num_sets
        return index, tag

    def read(self, address):
        index, tag = self._get_address_parts(address)
        target_set = self.sets[index]

        # 1. Procura por um HIT
        for i, line in enumerate(target_set):
            if line['valid'] == 1 and line['tag'] == tag:
                self.hits += 1
                target_set[0]['lru'], target_set[1]['lru'] = (1-i), i # Atualiza LRU
                return line['data']

        # 2. É um MISS
        self.misses += 1
        data = self.main_memory.load_word(address)

        # Escolhe a via a ser substituída (a que tem lru bit diferente do seu próprio índice)
        way_to_replace = next(i for i, line in enumerate(target_set) if line['lru'] != i)

        target_set[way_to_replace].update({'valid': 1, 'tag': tag, 'data': data})
        target_set[0]['lru'], target_set[1]['lru'] = (1-way_to_replace), way_to_replace # Atualiza LRU

        return data

    def write(self, address, data):
        index, tag = self._get_address_parts(address)
        target_set = self.sets[index]
        self.main_memory.store_word(address, data) # Write-Through

        for i, line in enumerate(target_set):
            if line['valid'] == 1 and line['tag'] == tag: # Write-Hit
                self.hits += 1
                line['data'] = data
                target_set[0]['lru'], target_set[1]['lru'] = (1-i), i # Atualiza LRU
                return

        self.misses += 1 # Write-Miss, No-Write-Allocate

    def get_stats(self):
        total = self.hits + self.misses
        if total == 0: return 0.0, 0.0, 0, 0
        hit_rate = (self.hits / total) * 100
        miss_rate = (self.misses / total) * 100
        return hit_rate, miss_rate, self.hits, self.misses

print(f"{LogColors.SUCCESS}Componente de Cache (Parte 5) carregado.{LogColors.RESET}")

Componente de Cache (Parte 5) carregado.


In [185]:
# ===================================================================
# Bloco 8: Integração Final (Pipeline + Cache)
#
# VERSÃO FINAL: Redefine a lógica do pipeline para que as instruções
# de memória (LW e SW) possam usar a Cache ou acessar a memória
# principal diretamente, com base na seleção do usuário.
# ===================================================================

def run_program_pipeline(asm_code, regs, mem, cache, hazard_mode='forwarding', cache_mode='enabled'):
    try: program = parse_program_single_cycle(asm_code)
    except ValueError as e: print(f"{LogColors.ERROR}ERRO DE SINTAXE: {e}{LogColors.RESET}"); return

    pc, cycle, stalls = 0, 0, 0
    pipeline_stages = {'IF': None, 'ID': None, 'EX': None, 'MEM_WB': None}
    history, hazard_logs = [], []

    while pc < len(program) or any(s is not None for s in pipeline_stages.values()):
        cycle += 1
        prev_stages = pipeline_stages.copy()
        stall_this_cycle = False

        # ESTÁGIO 4: MEMORY ACCESS & WRITE BACK (MEM_WB)
        instr_wb = prev_stages['MEM_WB']
        if instr_wb:
            op = instr_wb['op']

            ### --- CORREÇÃO AQUI: Lógica de acesso à memória condicional --- ###
            if op == "LW":
                address = instr_wb['addr']
                # Decide se usa a cache ou a memória principal
                if cache_mode == 'enabled':
                    instr_wb['result'] = cache.read(address)
                else: # Acesso direto à memória
                    instr_wb['result'] = mem.load_word(address)
                regs.write(instr_wb['rd'], instr_wb['result'])

            elif op == "SW":
                address = instr_wb['addr']
                data = regs.read(instr_wb['rs1'])
                # Decide se usa a cache ou a memória principal
                if cache_mode == 'enabled':
                    cache.write(address, data)
                else: # Acesso direto à memória
                    mem.store_word(address, data)

            elif 'result' in instr_wb: # Instruções Aritméticas
                regs.write(instr_wb['rd'], instr_wb['result'])
            ### --- FIM DA CORREÇÃO --- ###

        # ESTÁGIO 3: EXECUTE (EX)
        # ... (lógica inalterada) ...
        instr_ex = prev_stages['EX']
        if instr_ex:
            op = instr_ex['op']
            val1 = regs.read(instr_ex.get('rs1', 0))
            val2 = regs.read(instr_ex.get('rs2', 0)) if 'rs2' in instr_ex else instr_ex.get('imm', 0)
            if hazard_mode == 'forwarding':
                instr_mem = prev_stages['MEM_WB']
                if instr_mem and 'result' in instr_mem and 'rd' in instr_mem and instr_mem['op'] != 'SW':
                    fwd_reg = instr_mem['rd']
                    if fwd_reg == instr_ex.get('rs1'): val1 = instr_mem['result']; hazard_logs.append(f"{LogColors.INFO}Ciclo {cycle}:{LogColors.RESET} {LogColors.SUCCESS}FORWARDING (MEM->EX){LogColors.RESET} para reg x{fwd_reg} na instrução '{format_instruction(instr_ex)}'")
                    if 'rs2' in instr_ex and fwd_reg == instr_ex.get('rs2'): val2 = instr_mem['result']; hazard_logs.append(f"{LogColors.INFO}Ciclo {cycle}:{LogColors.RESET} {LogColors.SUCCESS}FORWARDING (MEM->EX){LogColors.RESET} para reg x{fwd_reg} na instrução '{format_instruction(instr_ex)}'")
            if op in ["ADD", "ADDI"]: instr_ex['result'] = to_signed32(val1) + to_signed32(val2)
            elif op == "SUB": instr_ex['result'] = to_signed32(val1) - to_signed32(val2)
            pipeline_stages['MEM_WB'] = instr_ex
        else: pipeline_stages['MEM_WB'] = None

        # ESTÁGIO 2: INSTRUCTION DECODE (ID)
        # ... (lógica inalterada) ...
        instr_id = prev_stages['ID']
        if instr_id:
            src_regs = {instr_id.get('rs1'), instr_id.get('rs2')} - {None}
            instr_ex_prev = prev_stages['EX']
            if instr_ex_prev and 'rd' in instr_ex_prev and instr_ex_prev['rd'] in src_regs:
                hazard_reg = instr_ex_prev['rd']
                if instr_ex_prev['op'] == 'LW':
                    if hazard_mode != 'none': stall_this_cycle = True; hazard_logs.append(f"{LogColors.INFO}Ciclo {cycle}:{LogColors.RESET} {LogColors.ERROR}HAZARD DE LOAD-USE detectado{LogColors.RESET} no reg x{hazard_reg}. {LogColors.WARNING}Ação: STALL OBRIGATÓRIO.{LogColors.RESET}")
                    else: hazard_logs.append(f"{LogColors.INFO}Ciclo {cycle}:{LogColors.RESET} {LogColors.ERROR}HAZARD DE LOAD-USE detectado{LogColors.RESET} no reg x{hazard_reg}. {LogColors.WARNING}Ação: Nenhuma.{LogColors.RESET}")
                else:
                    if hazard_mode == 'stall': stall_this_cycle = True; hazard_logs.append(f"{LogColors.INFO}Ciclo {cycle}:{LogColors.RESET} {LogColors.WARNING}HAZARD DE DADOS (EX->ID) detectado{LogColors.RESET} no reg x{hazard_reg}. {LogColors.WARNING}Ação: STALL.{LogColors.RESET}")
                    elif hazard_mode == 'none': hazard_logs.append(f"{LogColors.INFO}Ciclo {cycle}:{LogColors.RESET} {LogColors.ERROR}HAZARD DE DADOS (EX->ID) detectado{LogColors.RESET} no reg x{hazard_reg}. {LogColors.WARNING}Ação: Nenhuma.{LogColors.RESET}")

        if stall_this_cycle: stalls += 1; pipeline_stages['EX'] = None
        else:
            pipeline_stages['EX'] = prev_stages['ID']
            pipeline_stages['ID'] = prev_stages['IF']
            if pc < len(program): pipeline_stages['IF'] = program[pc]; pc += 1
            else: pipeline_stages['IF'] = None

        history.append({ 'Ciclo': cycle, 'IF': format_instruction(pipeline_stages['IF']), 'ID': format_instruction(pipeline_stages['ID']), 'EX': f"*{format_instruction(pipeline_stages['EX'])}" if stall_this_cycle else format_instruction(pipeline_stages['EX']), 'MEM_WB': format_instruction(pipeline_stages['MEM_WB'])})
        if cycle > (len(program) + stalls) * 2 and cycle > 30: print(f"{LogColors.WARNING}Simulação parada.{LogColors.RESET}"); break

    if hazard_logs: print(f"\n{LogColors.INFO}--- Detalhes de Hazards e Forwarding ---{LogColors.RESET}"); [print(log) for log in hazard_logs]
    print(f"\n{LogColors.INFO}--- Tabela de Execução do Pipeline ---{LogColors.RESET}")
    if history:
        df = pd.DataFrame(history).set_index('Ciclo')
        with output_area:
            display(df.fillna(""))

    print(f"\n{LogColors.INFO}Total de Ciclos (Pipeline):{LogColors.RESET} {cycle}\n{LogColors.INFO}Número de Stalls (bolhas):{LogColors.RESET} {stalls}\n{LogColors.INFO}Total de Ciclos (Ciclo Único seria):{LogColors.RESET} {len(program)}")

print(f"{LogColors.SUCCESS}Funções de simulação de Pipeline com Cache (Parte 5 - FINAL) carregadas.{LogColors.RESET}")

Funções de simulação de Pipeline com Cache (Parte 5 - FINAL) carregadas.
